In [23]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector
import numpy as np
import os
import tensorflow as tf
from encoder import CharacterTable
import roman_numerals as cnv
import random
import sklearn.model_selection
from sklearn.model_selection import train_test_split

In [62]:
OUTPUT_SEQUENCE_LEN=10

model = Sequential()
model.add(Dense(16, input_dim=1) )
model.add(RepeatVector(OUTPUT_SEQUENCE_LEN)) #length of the text
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(6,activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy','mae'])
num_epochs = 0

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                32        
                                                                 
 repeat_vector_2 (RepeatVec  (None, 10, 16)            0         
 tor)                                                            
                                                                 
 lstm_6 (LSTM)               (None, 10, 128)           74240     
                                                                 
 lstm_7 (LSTM)               (None, 10, 128)           131584    
                                                                 
 lstm_8 (LSTM)               (None, 10, 128)           131584    
                                                                 
 dense_5 (Dense)             (None, 10, 6)             774       
                                                      

In [72]:
DATASET_SIZE=201

samples = []
labels = []

import random

for i in range(DATASET_SIZE):
    samples.append(i)
    words = cnv.convert(i)
    labels.append(list(words))

samples = np.array(samples)
labels = np.array(labels, dtype=object)

print("Sample (input):",samples[3])
print("Label",labels[3])

codes = ' CLXVI'

nlabels = np.zeros((DATASET_SIZE,OUTPUT_SEQUENCE_LEN,len(codes)))
for i in range(DATASET_SIZE):
    for j in range(OUTPUT_SEQUENCE_LEN):
        if j>=len(labels[i]):
                nlabels[i][j][0]=1
                continue
        x = labels[i][j]
        index = codes.index(x)
        nlabels[i][j][index] = 1
print("Label encoded (output):\n",nlabels[3])
labels = nlabels
print(labels.shape)

Sample (input): 3
Label ['I', 'I', 'I']
Label encoded (output):
 [[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
(201, 10, 6)


In [60]:
print(labels[199])
TRAINING_SIZE = .5

(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
Training samples: 100  test samples 101


In [68]:
EPOCHS=50
BATCH_SIZE = int(len(trainSamples)/2)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
print("Epochs so far",num_epochs)
for x in range(100):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print()
    print("Epoch {} - loss ={:6.3f}, loss improvement ={:6.3f}".
          format(num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    pred = model.predict(samples)
    res = pred.argmax(axis=2)
    c,l,p = check_model()
#    print("accuracy={:6.3f}%".format(100*p))
#     f = open("output.txt", "a")
#     f.write("=================================================================================\n")
#     f.write("{} Epoch {} - loss ={:6.3f}, loss improvement ={:6.3f}\n".
#             format(i,num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
#     f.write("accuracy={:6.3f}%\n".format(100*p))
#     for i in range(len(pred)):
#         txt = label2words(res[i])
#         f.write("{} -> {}\n".format(i,txt))
#     f.close()
print("Done")


Training with 100 samples 50 epochs and batch_size= 50
Epochs so far 250

Epoch 300 - loss = 0.177, loss improvement =-0.000
7/7 [==============================] - 0s 15ms/step
69 -> LXII
173 -> CLXXIII
154 -> CLII
130 -> CXXXI
10 -> X
Correct 64 of 201  =  0.31840796019900497

Epoch 350 - loss = 0.245, loss improvement =-0.077
7/7 [==============================] - 0s 15ms/step
178 -> CLXXVII
46 -> XLVI
63 -> LXIII
83 -> LXXXIII
173 -> CLXVII
Correct 49 of 201  =  0.24378109452736318

Epoch 400 - loss = 0.202, loss improvement = 0.041
7/7 [==============================] - 0s 14ms/step
131 -> CXXXI
105 -> CII
161 -> CLXIII
118 -> CXVI
60 -> LXI
Correct 56 of 201  =  0.27860696517412936

Epoch 450 - loss = 0.309, loss improvement =-0.108
7/7 [==============================] - 0s 14ms/step
22 -> XXII
90 -> LCXI X
183 -> CLXXXVI
3 -> III
37 -> XXXVII
Correct 51 of 201  =  0.2537313432835821

Epoch 500 - loss = 0.170, loss improvement = 0.092
7/7 [==============================] - 0s 15ms

In [65]:
def label2words(label):
    s = ''
    for r in label:
        s+=codes[int(r)]
        #print(i,'->',s)
    return s.strip()

def check_model(verbose=0,show_training=1):
    pred = model.predict(samples)
    res = pred.argmax(axis=2)
    correct = 0
    for i in range(len(pred)):
        if(not show_training and i in trainSamples): continue
        train=''
        if i in trainSamples: train='[T]'
        txt = label2words(res[i])
        txt_correct = cnv.convert(i)
        ok=''
        if(txt==txt_correct):
            correct+=1
            ok = "[ok]"
        if(verbose==1):
            print(i,'->',txt, ok,train)
    if verbose==0:
        for i in range(5):
            x = random.randrange(DATASET_SIZE)
            print(x,'->',label2words(res[x]))
    print('Correct',correct,'of',len(pred),' = ',(correct/len(pred)))
    return correct,len(pred),(correct/len(pred))
check_model(1)

7/7 [==============================] - 0s 14ms/step
0 -> XXXII  
1 -> XII  
2 -> XI  
3 -> XI  [T]
4 -> XI  
5 -> XI  [T]
6 -> XI  [T]
7 -> XI  [T]
8 -> XI  [T]
9 -> XI  [T]
10 -> XI  [T]
11 -> XI [ok] [T]
12 -> XI  
13 -> XII  
14 -> XII  [T]
15 -> XXI  [T]
16 -> XXI  
17 -> XXI  [T]
18 -> XXI  
19 -> XXI  [T]
20 -> XXI  [T]
21 -> XXI [ok] 
22 -> XXI  [T]
23 -> XXI  
24 -> XXI  [T]
25 -> XXII  
26 -> XXII  [T]
27 -> XXII  [T]
28 -> XXII  
29 -> XXII  
30 -> XXII  [T]
31 -> XXII  [T]
32 -> XXII  [T]
33 -> XXII  
34 -> XXII  
35 -> XXII  [T]
36 -> XXXI  
37 -> XXXI  
38 -> XXXI  [T]
39 -> XXXI  [T]
40 -> XXXI  
41 -> XXXI  [T]
42 -> XXXI  [T]
43 -> XXXI  
44 -> XXXI  [T]
45 -> XXXI  [T]
46 -> XXXI  [T]
47 -> XXXI  
48 -> XXXI  [T]
49 -> XXXI  
50 -> XXXI  
51 -> XXXI  [T]
52 -> XXXI  
53 -> XXXI  [T]
54 -> XXXI  
55 -> XXXI  
56 -> XXXI  
57 -> XXXI  [T]
58 -> XXXI  [T]
59 -> XXXI  [T]
60 -> XXXI  [T]
61 -> XXXI  
62 -> XXXI  
63 -> XXXI  [T]
64 -> XXXI  
65 -> XXXI  
66 -> XXXI  
67 ->

(2, 201, 0.009950248756218905)

In [ ]:
# model.save('model_number2words.h5')

In [69]:
print(samples.shape)

(201,)


In [70]:
input=190
x = model.predict(np.array([input]))
v = np.argmax(x,axis=2)
#print(v.shape)
print(label2words(v.ravel()))

1/1 [==============================] - 0s 32ms/step
CXCI
